In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

2021-10-15 14:38:20.627414: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-15 14:38:20.627539: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Read train and test csv files
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")

or_test_df = test_df

In [3]:
# to have an overlook of data
train_df.head(n=2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [4]:
train_df = train_df.drop(['Name', 'Ticket', 'PassengerId', 'Fare'], axis=1)
test_df = test_df.drop(['Name', 'Ticket', 'PassengerId', 'Fare'], axis=1)

In [5]:
train_df['relatives'] = train_df['SibSp'] + train_df['Parch']
test_df['relatives'] = test_df['SibSp'] + test_df['Parch']

train_df = train_df.drop(['SibSp', 'Parch'], axis=1)
test_df = test_df.drop(['SibSp', 'Parch'], axis=1)

In [6]:
str_columns = ['Sex', 'Cabin', 'Embarked']
num_columns = ['Pclass', 'Age', 'relatives']

### Converting columns with string values to integers

In [7]:
def convert_to_str(df):
    for col in df.columns:
        if (df[col].dtype == np.int64 or df[col].dtype == np.float64):
            df[col] = df[col].fillna(df[col].mean())
        else:
            df[col] = df[col].fillna(method='ffill')
    return df

train_df = convert_to_str(train_df)
test_df = convert_to_str(test_df)

In [8]:
train_df['type'] = 'train'
test_df['type'] = 'test'

# Add a dummy SalePrice column to test dataframe to make number of columns equal 
test_df['Survived'] = train_df['Survived'].iloc[:418]

df = train_df.append(test_df, ignore_index=True)

In [9]:
for col in str_columns:
    one_hot = pd.get_dummies(df[col])

    replace_cols = {}
    for one_col in one_hot.columns:
        replace_cols[one_col] = f"{col}_{one_col}"
    one_hot = one_hot.rename(columns=replace_cols)

    df = df.drop(col, axis = 1)
    df = df.join(one_hot)

In [10]:
train_df = df[df['type'] == 'train']
test_df = df[df['type'] == 'test']

train_df = train_df.drop(['type'], axis=1)
test_df = test_df.drop(['type'], axis=1)

test_df = test_df.reset_index(drop=True)

### Scaling

In [11]:
scaler = MinMaxScaler()
scaler.fit(train_df[num_columns])

MinMaxScaler()

In [12]:
train_df[num_columns] = scaler.transform(train_df[num_columns])
test_df[num_columns] = scaler.transform(test_df[num_columns])

In [13]:
test_df = test_df.drop(['Survived'], axis=1)

In [14]:
train_labels = train_df['Survived']
train_data = train_df.drop(['Survived'], axis=1)

In [15]:
model = RandomForestClassifier()

In [16]:
model.fit(train_data, train_labels)

RandomForestClassifier()

In [17]:
predictions = model.predict(test_df)

In [18]:
test_df['Survived'] = predictions

In [19]:
# Create results dataframe
results = pd.DataFrame()
results['PassengerId'] = or_test_df['PassengerId']
results['Survived'] = test_df['Survived']

In [20]:
results.shape

(418, 2)

In [21]:
results.to_csv('submissions.csv', index=False)